<a href="https://colab.research.google.com/github/Walidsati/AAI_612O/blob/main/Week4/Notebook4.4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# AAI612: Deep Learning & its Applications

*Notebook 4.3: Graded Assignment: Mini Project I*

<a href="https://colab.research.google.com/github/harmanani/AAI612/blob/main/Week4/Notebook4.4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assessment

In this assessment, you will train a new model that is able to recognize fresh and rotten fruit. You will need to get the model to a validation accuracy of `92%` in order to pass the assessment, though we challenge you to do even better if you can. You will have the use the skills that you learned in the previous exercises. Specifically, we suggest using some combination of transfer learning, data augmentation, and fine tuning.

## The Dataset

In this exercise, you will train a model to recognize fresh and rotten fruits. Download the dataset from [Kaggle](https://www.kaggle.com/sriramr/fruits-fresh-and-rotten-for-classification). The dataset structure is in the `data/fruits` folder. There are 6 categories of fruits: fresh apples, fresh oranges, fresh bananas, rotten apples, rotten oranges, and rotten bananas. This will mean that your model will require an output layer of 6 neurons to do the categorization successfully. You'll also need to compile the model with `categorical_crossentropy`, as we have more than two categories.

![image.png](attachment:4c8c02c9-0cbe-4048-8d01-cdd5e3cf3fe6.png)<img src="https://github.com/harmanani/AAI612/blob/main/Week4/images/fruits.png?raw=1" style="width: 600px;">

## Load ImageNet Base Model

Start with a model pretrained on `ImageNet`. Load the model with the correct weights, set an input shape, and choose to remove the last layers of the model. Remember that images have three dimensions: a height, and width, and a number of channels. Because these pictures are in color, there will be three channels for red, green, and blue. We've filled in the input shape for you. This cannot be changed or the assessment will fail. If you need a reference for setting up the pretrained model, please take a look at [Notebook 4.2](https://github.com/harmanani/AAI612/blob/main/Week4/Notebook%204.2.ipynb) where we implemented transfer learning.

In [14]:
import ssl
from tensorflow import keras

ssl._create_default_https_context = ssl._create_unverified_context


base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

## Freeze Base Model

Next, we suggest freezing the base model. This is done so that all the learning from the ImageNet dataset does not get destroyed in the initial training.

In [15]:
# Freeze base model
base_model.trainable = False

## Add Layers to Model

Now it's time to add layers to the pretrained model. Pay close attention to the last dense layer and make sure it has the correct number of neurons to classify the different types of fruit.  You may add more layers than specified below.

In [16]:
# Create inputs with correct shape
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
outputs = keras.layers.Dense(6, activation = 'softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(inputs=inputs ,outputs=outputs)

In [17]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ vgg16 (Functional)                   │ (None, 7, 7, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │           3,078 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,717,766 (56.14 MB)

 Trainable params: 3,078 (12.02 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

## Compile Model

Now it's time to compile the model with loss and metrics options. Remember that we're training on a number of different categories, rather than a binary classification problem.

In [18]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## Augment the Data

If you'd like, try to augment the data to improve the dataset. There is also documentation for the [Keras ImageDataGenerator class](https://keras.io/api/preprocessing/image/#imagedatagenerator-class). This step is optional, but it may be helpful to get to 92% accuracy.

In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

## Load Dataset

Now it's time to load the train and validation datasets. Pick the right folders, as well as the right `target_size` of the images (it needs to match the height and width input of the model you've created).

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import os
print(os.listdir('/content/drive/MyDrive/fruits'))

['.DS_Store', 'train', 'valid']


In [22]:
train = '/content/drive/MyDrive/fruits/train'
valid = '/content/drive/MyDrive/fruits/valid'

In [23]:
# load and iterate training dataset
train_it = datagen.flow_from_directory(train,
                                       target_size=(224,224),
                                       color_mode='rgb',
                                       class_mode="categorical")
# load and iterate validation dataset
valid_it = datagen.flow_from_directory(valid,
                                      target_size=(224,224),
                                      color_mode='rgb',
                                      class_mode="categorical")

Found 249 images belonging to 6 classes.
Found 66 images belonging to 6 classes.


## Train the Model

Time to train the model! Pass the `train` and `valid` iterators into the `fit` function, as well as setting your desired number of epochs.

In [24]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples//train_it.batch_size,
          validation_steps=valid_it.samples//valid_it.batch_size,
          epochs=20)

Epoch 1/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.2232 - loss: 7.3345 - val_accuracy: 0.2031 - val_loss: 5.1088
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 314ms/step - accuracy: 0.2812 - loss: 3.9357 - val_accuracy: 0.2500 - val_loss: 4.0182
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.2772 - loss: 4.4633 - val_accuracy: 0.3125 - val_loss: 3.3328
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 240ms/step - accuracy: 0.5000 - loss: 1.9310 - val_accuracy: 0.4688 - val_loss: 2.6164
Epoch 5/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.5452 - loss: 2.4481 - val_accuracy: 0.4844 - val_loss: 1.6919
Epoch 6/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 264ms/step - accuracy: 0.6250 - loss: 1.5268 - val_accuracy: 0.5156 - val_loss: 1.8613
Epoch 7/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 15s 925ms/step - accuracy: 0.5899 - loss: 1.5312 - val_accuracy: 0.6406 - val_loss: 1.6840
Epoch 8/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 250ms/step - accuracy: 0.7600 - loss: 0.9229 - val_accuracy: 0.6406 - val_loss: 1.49

## Unfreeze Model for Fine Tuning

If you have reached 92% validation accuracy already, this next step is optional. If not, we suggest fine tuning the model with a very low learning rate.

In [25]:
# Unfreeze the base model
base_model.trainable = True

# Compile the model with a low learning rate
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [26]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples//train_it.batch_size,
          validation_steps=valid_it.samples//valid_it.batch_size,
          epochs=10)

Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - accuracy: 0.7758 - loss: 0.9593 - val_accuracy: 0.9219 - val_loss: 0.5192
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 585ms/step - accuracy: 0.9062 - loss: 0.2275 - val_accuracy: 0.8750 - val_loss: 0.4016
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.8822 - loss: 0.2759 - val_accuracy: 0.9375 - val_loss: 0.2520
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 232ms/step - accuracy: 0.9062 - loss: 0.1791 - val_accuracy: 0.8438 - val_loss: 0.3774
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 986ms/step - accuracy: 0.9593 - loss: 0.0922 - val_accuracy: 0.8750 - val_loss: 0.4786
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 407ms/step - accuracy: 0.8750 - loss: 0.4730 - val_accuracy: 0.9375 - val_loss: 0.2116
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 986ms/step - accuracy: 0.9332 - loss: 0.1766 - val_accuracy: 0.9219 - val_loss: 0.3277
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 238ms/step - accuracy: 0.9688 - loss: 0.0949 - val_accuracy: 0.8906 - val_loss: 0.2

## Evaluate the Model

Hopefully, you now have a model that has a validation accuracy of 92% or higher. If not, you may want to go back and either run more epochs of training, or adjust your data augmentation.

## Discussion